In [27]:
# import get_cluster_centres
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans as kmn

In [28]:
def generate_vocabulary(img_list):
    temp_list = []
    flag = True
    for img in img_list:

        features = get_cluster_centres.get_sift_feature_descriptors(img)
        temp_list.append(features)

    vocab_list = temp_list[0]

    for ftr in range(1, len(temp_list)):
        vocab_list = np.vstack((vocab_list, temp_list[ftr]))
        # print(vocab_list.shape)
        
    vocabulary = vocab_list
    return (vocab_list, temp_list)

In [4]:
def generate_clusters(vocab, n_clusters = 500):
    
    kmeans = kmn(n_clusters).fit(vocab)
    return kmeans


In [30]:
import pickle
f= open("Test.sift","rb")
dic=pickle.load(f)
f.close()
#print(len(list(dict.values())))


In [31]:
sv=[]
Names=[]
for k in list(dic):
    for j in list(dic[k]):
        sv.append(dic[k][j])
        Names.append([k,j])

In [32]:
k=np.vstack(sv)

In [33]:
kmeans=generate_clusters(k,n_clusters=4000)

/usr/local/lib/python3.5/dist-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=300 should be larger than k=4000. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.5/dist-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=300 should be larger than k=4000. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.5/dist-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=300 should be larger than k=4000. Setting it to 3*k
  init_size=init_size)


In [12]:
f= open("../files/centroids/KMeans_4000","wb")
pickle.dump(kmeans,f)
f.close()

In [13]:
from sklearn.decomposition import PCA
pca = PCA(2)  # project from  128 to 2 dimensions
projected = pca.fit_transform(np.vstack(sv))
#print(sv.shape)
print(projected.shape)

(1268814, 2)


In [14]:
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()
plt.scatter(projected[:, 0], projected[:, 1], edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('coolwarm', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
#plt.colorbar();


Text(0, 0.5, 'component 2')

In [40]:
print(len(Names))
[print(str(i)+"\t"+str(Names[i])+"\n") for i in range(15)]

15
0	['tr', 'hard_single_3.jpg']

1	['tr', 'test_1.jpg']

2	['tr', 'easy_single_3.jpg']

3	['tr', 'hard_multi_1.jpg']

4	['tr', 'hard_single_1.jpg']

5	['tr', 'easy_multi_1.jpg']

6	['tr', 'easy_single_1.jpg']

7	['tr', 'hard_multi_3.jpg']

8	['tr', 'hard_single_2.jpg']

9	['tr', 'test_3.jpg']

10	['tr', 'easy_multi_2.jpg']

11	['tr', 'easy_multi_3.jpg']

12	['tr', 'test_2.jpg']

13	['tr', 'easy_single_2.jpg']

14	['tr', 'hard_multi_2.jpg']



[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [41]:
def generate_features(temp_list, kmeans, n_clusters):
    
    t_length = len(temp_list)
    p_array = np.zeros((1,n_clusters))

    for i in range(0, t_length):

        features = temp_list[i]
        print(features.shape)
        (x1,y1)=features.shape
        t_array = np.zeros((1, n_clusters)) #an array for each image
        for j in range(x1):
            des = features[j]
            des = des.reshape(1, 128)
            ind = kmeans.predict(des)
            t_array[0][ind] += 1

        p_array = np.vstack((p_array, t_array))
    
    p_array = p_array[1:]
    return p_array

In [42]:
f= open("../files/centroids/KMeans_4000","rb")
kmeans=pickle.load(f)
f.close()

In [43]:
histo_grm=generate_features(sv,kmeans,4000)

(3686, 128)
(368, 128)
(40, 128)
(3661, 128)
(391, 128)
(113, 128)
(41, 128)
(399, 128)
(1816, 128)
(534, 128)
(46, 128)
(41, 128)
(376, 128)
(27, 128)
(1599, 128)


In [44]:
print(len(histo_grm))


15


In [45]:
hs=np.array(histo_grm)
np.save("../files/tset_Histogram_4000",hs)
hs.shape

(15, 4000)

In [46]:
from sklearn.feature_extraction.text import TfidfTransformer
tfr = TfidfTransformer(smooth_idf=False)
transf=tfr.fit_transform(hs)
k= transf.toarray()

/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:1242: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [47]:
print(k)

[[0.02107198 0.01404799 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.293064   0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01558556 0.01558556 0.         ... 0.         0.         0.        ]]


In [48]:
S=np.matmul(k,k.transpose())

In [49]:
S=S-np.identity(len(S))
print(np.argmax(S, axis=1), np.amax(S, axis=1))  

[ 3  0  6  0  7  4 10  4 14  0  6  4  8 11  8] [0.97142926 0.03123614 0.10903494 0.97142926 0.78095225 0.13267033
 0.19565676 0.78095225 0.88012265 0.05239109 0.19565676 0.19565878
 0.03216251 0.15248435 0.88012265]


In [50]:
f= open("../files/Tf-idf_4000","rb")
s=pickle.load(f)
pickle.dump(f,Tf-idf)
f.close()
print(s)

FileNotFoundError: [Errno 2] No such file or directory: '../files/Tf-idf_4000'

In [51]:
f= open("../files/TestNames_4000","wb")
pickle.dump(Names,f)
# pickle.load(f)
f.close()